In [67]:
%xmode plain
import glob
import cv2
import numpy as np

Exception reporting mode: Plain


In [68]:
suits_dict = {0:'Diamond',1:'Spade', 2:'Heart', 3:'Club'}

In [69]:
mask = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/04445-lbl.png',1)
img = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/04445.png',1)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(mask,127,255,0)
_,img = cv2.threshold(img,127,255,0)
im2 ,num_cards, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cimg = cv2.drawContours(img,num_cards,-1,(0,255,0),1)
# cv2.imshow('teg', cimg)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [70]:
def preprocess_card(contour, img):
    
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.09*peri, True)
    pts = np.float32(approx)
    x,y,w,h = cv2.boundingRect(contour)
    average = np.sum(pts, axis=0)/len(pts)
    
    warp = flattener(img, pts, w, h)
    
    #blur blur blur
#     cv2.imshow('ee',warp)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    ret,thresh = cv2.threshold(warp,127,255,cv2.THRESH_BINARY)
    im2 , contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    area = np.array([cv2.contourArea(contours[i], True) for i in range(len(contours))]) 
    print(len(contours))
    print(np.percentile(area[1:],50))
    suit_contours = (area>np.percentile(area[1:],50)).nonzero()[0]
    suit_areas = area[suit_contours]
    z = [x for _,x in sorted(zip(area[suit_contours],suit_contours))]
    largest_suit_contour = z[-2]
    x,y,w,h = cv2.boundingRect(contours[largest_suit_contour])
#     ret,th1 = cv2.threshold(warp,127,255,cv2.THRESH_BINARY)
    warp_c = cv2.rectangle(warp,(x,y),(x+w,y+h),(0,255,0),2)
    cropped_suit = warp_c[y:y+h, x:x+w]
    
    return cropped_suit, len(suit_contours)-1

In [71]:
def flattener(image, pts, w, h):
    """Flattens an image of a card into a top-down 200x300 perspective.
    Returns the flattened, re-sized, grayed image.
    See www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/"""
    temp_rect = np.zeros((4,2), dtype = "float32")
    
    s = np.sum(pts, axis = 2)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]

    diff = np.diff(pts, axis = -1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]

    # Need to create an array listing points in order of
    # [top left, top right, bottom right, bottom left]
    # before doing the perspective transform

    if w <= 0.8*h: # If card is vertically oriented
        temp_rect[0] = tl
        temp_rect[1] = tr
        temp_rect[2] = br
        temp_rect[3] = bl

    if w >= 1.2*h: # If card is horizontally oriented
        temp_rect[0] = bl
        temp_rect[1] = tl
        temp_rect[2] = tr
        temp_rect[3] = br

    # If the card is 'diamond' oriented, a different algorithm
    # has to be used to identify which point is top left, top right
    # bottom left, and bottom right.
    
    if w > 0.8*h and w < 1.2*h: #If card is diamond oriented
        # If furthest left point is higher than furthest right point,
        # card is tilted to the left.
        if pts[1][0][1] <= pts[3][0][1]:
            # If card is titled to the left, approxPolyDP returns points
            # in this order: top right, top left, bottom left, bottom right
            temp_rect[0] = pts[1][0] # Top left
            temp_rect[1] = pts[0][0] # Top right
            temp_rect[2] = pts[3][0] # Bottom right
            temp_rect[3] = pts[2][0] # Bottom left

        # If furthest left point is lower than furthest right point,
        # card is tilted to the right
        if pts[1][0][1] > pts[3][0][1]:
            # If card is titled to the right, approxPolyDP returns points
            # in this order: top left, bottom left, bottom right, top right
            temp_rect[0] = pts[0][0] # Top left
            temp_rect[1] = pts[3][0] # Top right
            temp_rect[2] = pts[2][0] # Bottom right
            temp_rect[3] = pts[1][0] # Bottom left
            
        
    maxWidth = 200
    maxHeight = 300

    # Create destination array, calculate perspective transform matrix,
    # and warp card image
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
    M = cv2.getPerspectiveTransform(temp_rect,dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)

    return warp

In [72]:
cropped_suit = 0
card_value = 0
for i in range(len(num_cards)):
    cropped_suit, card_value = preprocess_card(num_cards[i], img)

21
106.75


In [73]:
suits = [cv2.imread(file) for file in glob.glob("suits/*.png")]
for i,img in enumerate(suits):
    img = cv2.cvtColor(cv2.resize(img, (cropped_suit.shape[1], cropped_suit.shape[0])), cv2.COLOR_BGR2GRAY)
    suits[i] = img

In [74]:
sift = cv2.xfeatures2d.SIFT_create()
distances = []
for i in range(len(suits)):
    kp1, des1 = sift.detectAndCompute(cropped_suit, None)
    sift = cv2.xfeatures2d.SIFT_create()
    kp2, des2 = sift.detectAndCompute(suits[i], None)
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key = lambda x:x.distance)
    sums = 0
    for j in range(4):
        print(i," ",matches[j].distance)
        sums += matches[j].distance
    distances.append(sums)

print(distances)
print(" The card suit is:  ", suits_dict[np.argmin(np.array(distances))])

# # print(des1.shape)
# # print(des2.shape)

# img3  = np.array([])
# img3 = cv2.drawMatches(cropped_suit, kp1, cv2.flip(suits[2], 0), kp2, matches, cropped_suit, flags=2)


0   176.9717559814453
0   191.02879333496094
0   192.29664611816406
0   215.83326721191406
1   123.80226135253906
1   138.56045532226562
1   208.80374145507812
1   246.88661193847656
2   167.97023010253906
2   168.25279235839844
2   221.40460205078125
2   226.75096130371094
3   293.53363037109375
3   313.5091552734375
3   342.14178466796875
3   343.40936279296875
[776.1304626464844, 718.0530700683594, 784.3785858154297, 1292.5939331054688]
 The card suit is:   Spade


In [75]:
def find_rank(img_suit):
    _,img_suit = cv2.threshold(img_suit,127,255,0)
    suits = [cv2.imread(file) for file in glob.glob("suits/*.png")]
    for i,img in enumerate(suits):
        img = cv2.cvtColor(cv2.resize(img, (img_suit.shape[1], img_suit.shape[0])), cv2.COLOR_BGR2GRAY)
        _,img = cv2.threshold(img,127,255,0)
        suits[i] = img
    
    minimum1 = 1000000
    idx1  = 0   
    ret = 0
    for i in range(len(suits)):
        ans = np.abs((-img_suit + cv2.flip(suits[i], 0)))
        tans = ans[ans > 128].shape[0]
        print(i," ", tans)
        
        if tans < minimum1:
            minimum1 = tans
            idx1 = i
            ret = ans
    minimum2 = 1000000
    idx2 = 0


    for j in range(len(suits)):
        ans = np.abs(-img_suit + suits[j])
        ans = ans[ans > 128].shape[0]
        print(j," - ", ans)
        if ans < minimum2:
            minimum2 = ans
            idx2 = j
    cv2.imshow('answer ', np.abs(-img_suit + suits[0]))
    cv2.imshow('answer1 ', np.abs(-img_suit + suits[1]))
    cv2.imshow('answer2 ', np.abs(-img_suit + suits[2]))
    cv2.imshow('answer3 ', np.abs(-img_suit + suits[3]))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    if minimum2 < minimum1:   
        return idx2
    else:    
        return idx1

In [76]:
a = find_rank(cropped_suit)
print("The suit of the card is: ", suits_dict[a])
print("------ SIFT ------")
print(np.array(distances))
print(" The card suit is:  ", suits_dict[np.argmin(np.array(distances))])


0   443
1   255
2   479
3   329
0  -  381
1  -  473
2  -  107
3  -  549
The suit of the card is:  Heart
------ SIFT ------
[ 776.13046265  718.05307007  784.37858582 1292.59393311]
 The card suit is:   Spade
